In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = optim.SGD(model.parameters(), lr=2e-2)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step()

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().cpu().numpy().item()))

epoch:  0, loss: 0.38649848103523254
epoch:  1, loss: 0.35312575101852417
epoch:  2, loss: 0.32293909788131714
epoch:  3, loss: 0.29563575983047485
epoch:  4, loss: 0.27096229791641235
epoch:  5, loss: 0.2488178014755249
epoch:  6, loss: 0.22895927727222443
epoch:  7, loss: 0.21101608872413635
epoch:  8, loss: 0.19470717012882233
epoch:  9, loss: 0.17985931038856506
epoch:  10, loss: 0.16633912920951843
epoch:  11, loss: 0.1540299355983734
epoch:  12, loss: 0.14282579720020294
epoch:  13, loss: 0.13263137638568878
epoch:  14, loss: 0.12335719913244247
epoch:  15, loss: 0.11492391675710678
epoch:  16, loss: 0.10725942254066467
epoch:  17, loss: 0.10029556602239609
epoch:  18, loss: 0.0939723327755928
epoch:  19, loss: 0.08823325484991074
epoch:  20, loss: 0.08302566409111023
epoch:  21, loss: 0.07830239832401276
epoch:  22, loss: 0.07402035593986511
epoch:  23, loss: 0.07013946771621704
epoch:  24, loss: 0.066623255610466
epoch:  25, loss: 0.06343888491392136
epoch:  26, loss: 0.0605561

In [6]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = -2066.6998586749505
Test metrics:  R2 = -1994.3515784509143


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = optim.Adam(model.parameters(), lr=1e-2)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step()

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].detach().cpu().numpy().item()))

epoch:  0, loss: 0.04847664758563042
epoch:  1, loss: 0.035514701157808304
epoch:  2, loss: 0.03273080661892891
epoch:  3, loss: 0.03723014518618584
epoch:  4, loss: 0.036663856357336044
epoch:  5, loss: 0.03359885513782501
epoch:  6, loss: 0.03178165853023529
epoch:  7, loss: 0.03187888488173485
epoch:  8, loss: 0.0327356792986393
epoch:  9, loss: 0.03324923664331436
epoch:  10, loss: 0.03297702223062515
epoch:  11, loss: 0.03205196559429169
epoch:  12, loss: 0.030920229852199554
epoch:  13, loss: 0.03010989911854267
epoch:  14, loss: 0.02990950085222721
epoch:  15, loss: 0.030062012374401093
epoch:  16, loss: 0.029998019337654114
epoch:  17, loss: 0.029340120032429695
epoch:  18, loss: 0.028260523453354836
epoch:  19, loss: 0.02719656005501747
epoch:  20, loss: 0.026400625705718994
epoch:  21, loss: 0.02576613612473011
epoch:  22, loss: 0.024950813502073288
epoch:  23, loss: 0.02371724136173725
epoch:  24, loss: 0.022224897518754005
epoch:  25, loss: 0.020941222086548805
epoch:  26, 

In [8]:
pred_train = model.forward(X_train).detach()
pred_test = model.forward(X_test).detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train)}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test)}")

Train metrics: R2 = 0.7083075638486611
Test metrics:  R2 = 0.7475086476072998
